In [1]:
from pyspark import	SparkConf, SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

## Adicionando os Dados

In [2]:
# iniciando uma sessão 
ssql = SparkSession.builder.appName("Client").getOrCreate()

In [7]:
# Carregando o dataset utilizado
data = ssql.read.csv('Data.csv', header=True)

# Carregando o dataset utilizado
ssql.read.csv('Data.csv', header=True).createOrReplaceTempView("Data")

In [8]:
data.show()

+---+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+---------------+---------+--------------------+------+-------------+--------------------+--------------------+
|_c0|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page|   registration|sessionId|                song|status|           ts|           userAgent|              userId|
+---+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+---------------+---------+--------------------+------+-------------+--------------------+--------------------+
|  0|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000.0|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|                  30|
|  1|    Fiv

In [ ]:
print(f'O data-set possui {data.count()} fileiras')
print(f'e {len(data.columns)} colunas')

AttributeError: 'NoneType' object has no attribute 'count'

In [ ]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: string (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: string (nullable = true)
 |-- sessionId: string (nullable = true)
 |-- song: string (nullable = true)
 |-- status: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



## EDA

In [ ]:
# Loop para contabilizar quantia de nulls em cada coluna
for i in data.columns[1:]:
    nulls = data.filter(f'{i} is NULL').count()
    print(f'Quantia de Nulls na coluna {i} ', nulls)

Quantia de Nulls na coluna artist  58392
Quantia de Nulls na coluna auth  0
Quantia de Nulls na coluna firstName  8346
Quantia de Nulls na coluna gender  8346
Quantia de Nulls na coluna itemInSession  0
Quantia de Nulls na coluna lastName  8346
Quantia de Nulls na coluna length  58392
Quantia de Nulls na coluna level  0
Quantia de Nulls na coluna location  8346
Quantia de Nulls na coluna method  0
Quantia de Nulls na coluna page  0
Quantia de Nulls na coluna registration  8346
Quantia de Nulls na coluna sessionId  0
Quantia de Nulls na coluna song  58392
Quantia de Nulls na coluna status  0
Quantia de Nulls na coluna ts  0
Quantia de Nulls na coluna userAgent  8346
Quantia de Nulls na coluna userId  8346


Podemos perceper que há grande quantia de nulls, e que há pelo menos 3 colunas com as mesmas quantias, provavelmente se uma determinada coluna estiver com null as outras também estarão

In [ ]:
ssql.sql("select artist, length, song "+
         "from data "+
         "where artist is null and "+
         "length is null and "+
         "song is null").count()

In [ ]:
ssql.sql("select firstName, gender, lastName, location, registration, userAgent, userID "+
         "from data "+
         "where gender is null and firstName is null and lastName is null and registration is null and "+
         "userAgent is null and userID is null").count()

Como podemos perceber os valores que faltam podem seguir um certo padrão já que seguem uma sequência, esse problema será resolvido nas seguintes partes levando essa informação em conta, pois irei usar uma forma específica para lidar com os valores faltantes